In [104]:
import numpy as np

np.random.seed(666)

In [105]:
# Parámetros
S0 = 50
K = 52
r = 0.06
sigma = 0.25
T = 1/12  # 1 mes en años
n_pasos = 22 # días de trading aprox en 1 mes
n_sim = 50000 # simulaciones requeridas
dt = T / n_pasos


#  Simulación Monte Carlo Estándar
Z = np.random.normal(size=(n_sim, n_pasos))  # variables aleatorias normales
paths = np.zeros((n_sim, n_pasos+1))  # matriz para almacenar las trayectorias
paths[:, 0] = S0  # precio inicial de cada simulación

for i in range(1, n_pasos+1):
    paths[:, i] = paths[:, i-1] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z[:, i-1])  # fórmula de GBM para cada paso

S_avg = np.mean(paths[:, 1:], axis=1)  # precio promedio del activo subyacente por simulación
payoff_call = np.maximum(S_avg - K, 0)
price_MC = np.exp(-r*T) * np.mean(payoff_call)

#  Antitéticas
Z2 = np.random.normal(size=(n_sim//2, n_pasos))  # variables aleatorias normales para la mitad de simulaciones, la otra mitad será antitética
paths1 = np.zeros((n_sim//2, n_pasos+1))
paths2 = np.zeros((n_sim//2, n_pasos+1))
paths1[:,0] = S0
paths2[:,0] = S0

for i in range(1, n_pasos+1):
    paths1[:,i] = paths1[:,i-1] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z2[:,i-1])
    paths2[:,i] = paths2[:,i-1] * np.exp((r - 0.5*sigma**2)*dt - sigma*np.sqrt(dt)*Z2[:,i-1])

S_avg1 = np.mean(paths1[:,1:], axis=1)
S_avg2 = np.mean(paths2[:,1:], axis=1)

payoff1 = np.maximum(S_avg1 - K, 0)
payoff2 = np.maximum(S_avg2 - K, 0)

price_anti = np.exp(-r*T) * np.mean(np.concatenate([payoff1, payoff2]))  # juntar ambas mitades

print(f"(b) Precio Call Asiática (MC estándar) = {price_MC:.4f}")
print(f"(c) Precio Call Asiática (antitéticas) = {price_anti:.4f}")

(b) Precio Call Asiática (MC estándar) = 0.2409
(c) Precio Call Asiática (antitéticas) = 0.2405
